# Statement

A companhia de seguros Proteja Seu Amanhã quer resolver algumas tarefas com a ajuda de aprendizado de máquina e você precisa avaliar a possibilidade de fazê-lo.

- Tarefa 1: Encontrar clientes semelhantes a um determinado cliente. Isso vai ajudar os agentes da empresa com tarefas de marketing.
- Tarefa 2: Predizer se um novo cliente provavelmente receberá um pagamento de seguro. Um modelo de predição pode ser melhor do que um modelo dummy?
- Tarefa 3: Predizer o número de pagamentos de seguro que um novo cliente provavelmente receberá usando um modelo de regressão linear.
- Tarefa 4: Proteger os dados pessoais dos clientes sem estragar o modelo da tarefa anterior. É necessário desenvolver um algoritmo de transformação de dados que tornaria difícil recuperar informações pessoais se os dados caíssem nas mãos erradas. Isso é chamado de mascaramento de dados ou ofuscação de dados. Mas os dados devem ser protegidos de forma que a qualidade dos modelos de aprendizado de máquina não piore. Você não precisa escolher o melhor modelo, só prove que o algoritmo funciona corretamente.

# Pré-processamento de dados & Exploração

## Inicialização

In [1]:
pip install scikit-learn --upgrade


[notice] A new release of pip available: 22.3.1 -> 23.2.1
[notice] To update, run: python.exe -m pip install --upgrade pip



     ---------------------------------------- 9.2/9.2 MB 9.8 MB/s eta 0:00:00
     --------------------------------------- 15.5/15.5 MB 27.3 MB/s eta 0:00:00
     --------------------------------------- 44.0/44.0 MB 50.4 MB/s eta 0:00:00
     ---------------------------------------- 302.2/302.2 kB ? eta 0:00:00


In [2]:
import numpy as np
import pandas as pd
import plotly.express as px
from sklearn.metrics import r2_score
import seaborn as sns
import math
import sklearn.metrics
import sklearn.linear_model
import sklearn.metrics
import sklearn.neighbors
import sklearn.preprocessing
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neighbors import NearestNeighbors
from sklearn.model_selection import train_test_split
from scipy.spatial import distance
from sklearn.ensemble import RandomForestRegressor

from IPython.display import display

ModuleNotFoundError: No module named 'pandas'

## Carregar Dados

Carregue os dados e faça uma verificação básica de que estão livres de problemas óbvios.

In [ ]:
df = pd.read_csv('/datasets/insurance_us.csv')

Renomeamos as colunas para tornar o código mais consistente com seu estilo.

In [ ]:
df = df.rename(columns={'Gender': 'gender', 'Age': 'age', 'Salary': 'income', 'Family members': 'family_members', 'Insurance benefits': 'insurance_benefits'})

In [ ]:
df.sample(10)

In [ ]:
df.info()

In [ ]:
# podemos querer corrigir o tipo de idade (de float para int), embora isso não seja crítico

# escreva sua conversão aqui se você escolher:

df['age'] = df['age'].astype('int')

In [ ]:
# verifique se a conversão foi bem-sucedida
df.info()

In [ ]:
# agora dê uma olhada nas estatísticas descritivas dos dados.
# Parece que está tudo bem?

In [ ]:
df.describe()

* Aparentemente a coluna insurance_benefits tem muitos outliars

In [ ]:
df.duplicated().sum()

In [ ]:
df =  df.drop_duplicates().reset_index(drop=True)

In [ ]:
df.duplicated().sum()

* Utilizei o drop_duplicated pois não 153 linhas são insignificante em um dataframe de 5000 linhas

## AED

Vamos verificar rapidamente se existem determinados grupos de clientes observando o gráfico de pares.

In [ ]:
g = sns.pairplot(df, kind='hist')
g.fig.set_size_inches(12, 12)

In [ ]:
px.scatter_3d(df,x='age',y='income',z='insurance_benefits', color = 'family_members')

* Podemos ver uma relação em que o grupo que mais recebe insurance benefits é o grupo mais velho acima de 40 anos, com uma relação de estreitamento em direção a 'média' com o income, conforme a idade aumenta os selecionados estão mais no meio ainda. E com poucos membros na família. Obviamente levando em conta uma igualdade de gêneros.

In [ ]:
df['gender'].value_counts()

* O que se confirma

Ok, é um pouco difícil identificar grupos óbvios (clusters), pois é difícil combinar várias variáveis simultaneamente (para analisar distribuições multivariadas). É aí que Álgebra Linear e Aprendizado de Máquina podem ser bastante úteis.

# Tarefa 1. Clientes Similares

Na linguagem de AM, é necessário desenvolver um procedimento que retorne k vizinhos mais próximos (objetos) para um determinado objeto com base na distância entre os objetos.
Você pode querer rever as seguintes lições (capítulo -> lição)- Distância Entre Vetores -> Distância Euclidiana
- Distância Entre Vetores -> Distância de Manhattan

Para resolver a tarefa, podemos tentar diferentes métricas de distância.

Escreva uma função que retorne k vizinhos mais próximos para um n-ésimo objeto com base em uma métrica de distância especificada. O número de pagamentos de seguro recebidos não deve ser levado em consideração para esta tarefa. 

Você pode usar uma implementação pronta do algoritmo kNN do scikit-learn (verifique [o link](https://scikit-learn.org/stable/modules/generated/sklearn.neighbors.NearestNeighbors.html#sklearn.neighbors.NearestNeighbors)) ou usar a sua própria.
Teste-o para quatro combinações de dois casos
- Escalabilidade
  - os dados não são escalados
  - os dados escalados com o escalonador [MaxAbsScaler](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.MaxAbsScaler.html) 
- Métricas de distância
  - Euclidiana
  - Manhattan

Responda às perguntas:
- Os dados não escalados afetam o algoritmo kNN? Se sim, como isso acontece?
-Quão semelhantes são os resultados usando a métrica de distância de Manhattan (independentemente da escalabilidade)?

In [ ]:
feature_names = ['gender', 'age', 'income', 'family_members']

In [ ]:
def get_knn(df, n, k, metric):
    
    """
    Retorna os vizinhos mais próximos de k

    :param df: DataFrame pandas usado para encontrar objetos semelhantes dentro de    :param n: número do objeto pelo qual os vizinhos mais próximos são procurados
    :param k: o número dos vizinhos mais próximos a serem retornados
    :param metric: nome da métrica de distância    """
    
    if metric == 'euclidean':
        nbrs = NearestNeighbors(n_neighbors=k, metric='euclidean')
    elif metric == 'manhattan':
        nbrs = NearestNeighbors(n_neighbors=k, metric='manhattan')
    else:
        raise ValueError("Métrica não suportada. Escolha 'euclidean' ou 'manhattan'")

   
    nbrs = nbrs.fit(df[feature_names])
   
    nbrs_distances, nbrs_indices = nbrs.kneighbors([df.iloc[n][feature_names]], k, return_distance=True)
    
    df_res = pd.concat([
        df.iloc[nbrs_indices[0]], 
        pd.DataFrame(nbrs_distances.T, index=nbrs_indices[0], columns=['distance'])
        ], axis=1)
    
    return df_res

Escalando os dados

In [ ]:
feature_names = ['gender', 'age', 'income', 'family_members']

transformer_mas = sklearn.preprocessing.MaxAbsScaler().fit(df[feature_names].to_numpy())

df_scaled = df.copy()
df_scaled.loc[:, feature_names] = transformer_mas.transform(df[feature_names].to_numpy())

In [ ]:
df_scaled.sample(5)

Agora, vamos obter registros semelhantes para um determinado registro para cada combinação

In [ ]:
get_knn(df, 1, 10, 'manhattan')

In [ ]:
get_knn(df, 1, 10, 'euclidean')

In [ ]:
get_knn(df_scaled, 1, 10, 'euclidean')

In [ ]:
get_knn(df_scaled, 1, 10, 'manhattan')

Respostas para as perguntas

**Os dados não escalados afetam o algoritmo kNN? Se sim, como isso acontece?** 

Afetam sim, deixando a distância calculada maior.

As diferenças de escala podem levar a uma maior influência de algumas características em relação a outras. Por exemplo, se uma característica tiver valores muito maiores em comparação com outras, ela dominará as distâncias calculadas pelo kNN, mesmo que outras características sejam mais relevantes para a tarefa.

**Quão semelhantes são os resultados usando a métrica de distância de Manhattan (independentemente da escalabilidade)?** 

Bem semelhantes, mas maiores que a distância euclidiana

# Tarefa 2. É provável que o cliente receba um pagamento do seguro?

Em termos de aprendizado de máquina, podemos olhar para isso como uma tarefa de classificação binária.

Com os pagamentos de seguro sendo mais do que zero como objetivo, avalie se a abordagem da classificação kNN pode ser melhor do que um modelo dummy.

Instruções:
- Construa um classificador baseado em kNN e meça sua qualidade com a métrica F1 para k=1..10 tanto para os dados originais quanto para os escalados. Seria interessante ver como k pode influenciar a métrica de avaliação e se a escalabilidade dos dados faz alguma diferença. Você pode usar uma implementação pronta do algoritmo de classificação kNN do scikit-learn (verifique [o link](https://scikit-learn.org/stable/modules/generated/sklearn.neighbors.KNeighborsClassifier.html)) ou usar a sua própria.
- Construa o modelo dummy, que é aleatório para este caso. Deve retornar com alguma probabilidade o valor "1". LVamos testar o modelo com quatro valores de probabilidade: 0, a probabilidade de fazer qualquer pagamento de seguro, 0,5, 1.

A probabilidade de fazer qualquer pagamento de seguro pode ser definida como

$$
P\{\text{pagamento de seguro recebido}= número de clientes que receberam qualquer pagamento de seguro}}{\text{número total de clientes}}.
$$

Divida os dados inteiros na proporção 70:30 para as partes de treinamento/teste.

In [ ]:
def update(insurance_benefits):
    if insurance_benefits >= 1:
        return 1
    else:
        return 0

df['insurance_benefits_received'] = df['insurance_benefits'].apply(update)
df_scaled['insurance_benefits_received'] = df_scaled['insurance_benefits'].apply(update)

In [ ]:
# verifique o desequilíbrio de classe com value_counts()

display(df['insurance_benefits_received'].value_counts())
df_scaled['insurance_benefits_received'].value_counts()

In [ ]:
df_train, df_test = train_test_split(df, test_size=0.3, random_state=12345)

df_train2, df_test2 = train_test_split(df_scaled, test_size=0.3, random_state=12345)

In [ ]:
features_train = df_train.drop(['insurance_benefits_received','insurance_benefits'], axis=1)
target_train = df_train['insurance_benefits_received']

features_test = df_test.drop(['insurance_benefits_received','insurance_benefits'], axis=1)
target_test = df_test['insurance_benefits_received']

features_train2 = df_train2.drop(['insurance_benefits_received','insurance_benefits'], axis=1)
target_train2 = df_train2['insurance_benefits_received']

features_test2 = df_test2.drop(['insurance_benefits_received','insurance_benefits'], axis=1)
target_test2 = df_test2['insurance_benefits_received'] 


print(features_train.shape)
print(target_train.shape)
print(features_test.shape)
print(target_test.shape)


print(features_train2.shape)
print(target_train2.shape)
print(features_test2.shape)
print(target_test2.shape)

In [ ]:
for i in range(1,11):
    model = KNeighborsClassifier(n_neighbors=i)
    model.fit(features_train, target_train)
    test_predictions = model.predict(features_test)

    model2 = KNeighborsClassifier(n_neighbors=i)
    model2.fit(features_train2, target_train2)
    test_predictions2 = model2.predict(features_test2)

In [ ]:
def eval_classifier(y_true, y_pred):
    
    f1_score = sklearn.metrics.f1_score(y_true, y_pred)
    print(f'F1: {f1_score:.2f}')
    
# se você tiver um problema com a linha a seguir, reinicie o kernel e execute o caderno novamente
    cm = sklearn.metrics.confusion_matrix(y_true, y_pred, normalize='all')
    print('Matriz de Confusão')
    print(cm)

In [ ]:
# gerando saída de um modelo aleatório

def rnd_model_predict(P, size, seed=42):

    rng = np.random.default_rng(seed=seed)
    return rng.binomial(n=1, p=P, size=size)

In [ ]:
for P in [0, df['insurance_benefits_received'].sum() / len(df), 0.5, 1]:

    print(f'A probabilidade: {P:.2f}')
    y_pred_rnd = rnd_model_predict(P, size=len(df['insurance_benefits_received']))
        
    eval_classifier(df['insurance_benefits_received'], y_pred_rnd)
    
    print()

In [ ]:
for i in range(1,11):
    print('k=',i)
    model = KNeighborsClassifier(n_neighbors=i)
    model.fit(features_train, target_train)
    test_predictions = model.predict(features_test)
    eval_classifier(target_test, test_predictions)
    print('k scaled=',i)
    model2 = KNeighborsClassifier(n_neighbors=i)
    model2.fit(features_train2, target_train2)
    test_predictions2 = model2.predict(features_test2)
    eval_classifier(target_test2, test_predictions2)
    

* O maior valor f1 encontrado foi com o parâmetro k=1 com os dados *scaled*

# Tarefa 3. Regressão (com Regressão Linear)

Com os pagamentos de seguro como objetivo, avalie qual seria o REQM para um modelo de Regressão Linear.

Construa sua própria implementação de Regressão Linear. Para isso, lembre-se de como a solução da tarefa de regressão linear é formulada em termos de Álgebra linear. Verifique o REQM para os dados originais e os escalados. Você pode ver alguma diferença no REQM entre esses dois casos?

Vamos denotar
- $X$ — matriz de características, cada linha é um caso, cada coluna é uma característica, a primeira coluna consiste em unidades
- $y$ — objetivo (um vetor)
- $\hat{y}$ — objetivo estimado (um vetor)- $w$ — vetor de peso

A tarefa de regressão linear na linguagem de matrizes pode ser formulada como
$$
y = Xw
$$

O objetivo do treinamento, então, é encontrar os $w$ que minimizaria a distância L2 (EQM) entre $Xw$ e $y$:

$$
\min_w d_2(Xw, y) \quad \text{or} \quad \min_w \text{MSE}(Xw, y)
$$

Parece que há uma solução analítica para a questão acima:

$$
w = (X^T X)^{-1} X^T y
$$

A fórmula acima pode ser usada para encontrar os pesos $w$ e o último pode ser usado para calcular valores preditos

$$
\hat{y} = X_{val}w
$$

Divida todos os dados na proporção 70:30 para as partes de treinamento/validação. Use a métrica REQM para a avaliação do modelo.

In [ ]:


class MyLinearRegression:
    
    def __init__(self):
        self.weights = None
    
    def fit(self, X, y):
        # Adicionando uma coluna de uns para representar o termo de bias
        X2 = np.append(np.ones([len(X), 1]), X, axis=1)
        
        # Calculando os pesos usando a fórmula dos mínimos quadrados
        self.weights = np.linalg.inv(X2.T @ X2) @ X2.T @ y

    def predict(self, X):
        # Adicionando uma coluna de uns para representar o termo de bias
        X2 = np.append(np.ones([len(X), 1]), X, axis=1)
        
        # Realizando a previsão usando os pesos aprendidos
        y_pred = X2 @ self.weights
        
        return y_pred


In [ ]:
def eval_regressor(y_true, y_pred):
    
    rmse = math.sqrt(sklearn.metrics.mean_squared_error(y_true, y_pred))
    print(f'REQM: {rmse:.2f}')
    
    r2_score = math.sqrt(sklearn.metrics.r2_score(y_true, y_pred))
    print(f'R2: {r2_score:.2f}')    

In [ ]:
Xx = df[['age', 'gender', 'income', 'family_members']].to_numpy()
y = df['insurance_benefits'].to_numpy()

X_train, X_test, y_train, y_test = train_test_split(Xx, y, test_size=0.3, random_state=12345)

lr = MyLinearRegression()

lr.fit(X_train, y_train)
print(lr.weights)

y_test_pred = lr.predict(X_test)
eval_regressor(y_test, y_test_pred)

In [ ]:
Xxx = df_scaled[['age', 'gender', 'income', 'family_members']].to_numpy()
yx = df_scaled['insurance_benefits'].to_numpy()

X_trainx, X_testx, y_trainx, y_testx = train_test_split(Xxx, yx, test_size=0.3, random_state=12345)

lrx = MyLinearRegression()

lrx.fit(X_trainx, y_trainx)
print(lrx.weights)

y_test_predx = lrx.predict(X_testx)
eval_regressor(y_testx, y_test_predx)

* Sem diferença no REQM nesses dois casos.


# Tarefa 4. Ofuscando dados

É melhor ofuscar os dados multiplicando as características numéricas (lembre-se, elas podem ser vistos como a matriz $X$) por uma matriz invertível $P$. 

$$
X' = X \times P
$$

Tente fazer isso e verifique como os valores das características ficarão após a transformação. Aliás, a invertibilidade é importante aqui, portanto, certifique-se de que $P$ seja realmente invertível.

Você pode querer revisar a lição 'Matrizes e Operações com Matrizes -> Multiplicação de Matrizes' para relembrar a regra de multiplicação de matrizes e sua implementação com NumPy.

In [ ]:
personal_info_column_list = ['gender', 'age', 'income', 'family_members']
df_pn = df[personal_info_column_list]

In [ ]:
#matriz original
X = df_pn.to_numpy()
X

Gerando uma matriz $P$ aleatória.

In [ ]:
# matriz aleatoria
rng = np.random.default_rng(seed=42)
P = rng.random(size=(X.shape[1], X.shape[1]))

Verificando se a matriz $P$ é invertível

In [ ]:
p1=np.linalg.inv(P)

Você consegue adivinhar a idade ou a renda dos clientes após a transformação?
* Não

In [ ]:
#matriz transformada
X1=X @ P
X1

Você pode recuperar os dados originais de $X′$ se souber $P$? Tente verificar isso com cálculos movendo $P$ do lado direito da fórmula acima para o esquerdo. As regras da multiplicação de matrizes são realmente úteis aqui

In [ ]:

XX=X1 @ p1
XX

Imprima todos os três casos para alguns clientes- Os dados originais
- O transformado
- O invertido (recuperado)

In [ ]:
display(X,X1,XX)


Você provavelmente pode ver que alguns valores não são exatamente iguais aos dos dados originais. Qual pode ser a razão disso?

* Acredito que se deve à natureza das transformações aplicadas e à complexidade dos algoritmos envolvidos. Por se tratar de uma matriz aleatória na ofuscação e a inversa dela na recuperação. 

## Provas de que a ofuscação de dados pode funcionar com a Regressão Linear

A tarefa de regressão foi resolvida com regressão linear neste projeto. Sua próxima tarefa é provar analiticamente que o método de ofuscação fornecido não afetará a regressão linear em termos de valores preditos, ou seja, seus valores permanecerão os mesmos. Você acredita nisso? Bem, você não precisa acreditar, você deve provar isso!

Assim, os dados são ofuscados e há $X \ P$ em vez de apenas X agora. Consequentemente, existem outros pesos $w_P$ como
$$
w = (X^T X)^{-1} X^T y \quad \Rightarrow \quad w_P = [(XP)^T XP]^{-1} (XP)^T y
$$

Como  $w$ e $w_P$ seriam ligados se você simplificasse a fórmula para $w_P$ acima? 

Quais seriam os valores previstos com $w_P$? 

O que isso significa para a qualidade da regressão linear se você medir com REQM?

Verifique o Apêndice B Propriedades das Matrizes no final do caderno. Existem fórmulas úteis lá!

Nenhum código é necessário nesta seção, apenas explicação analítica!

**Resposta**

1- Para simplificar essa fórmula, podemos usar a propriedade das matrizes que diz que <td>$(AB)^T = B^TA^T$</td>. Aplicando isso, podemos reescrever 𝑤𝑃 como <td>$$𝑤𝑃=(𝑋𝑃)^{-1}(𝑋𝑃)^𝑇𝑦$$</td>.

2- Para calcular os valores previstos com 𝑤𝑃, você multiplica a matriz de características 𝑋𝑃 pelos pesos 𝑤𝑃. Ou seja, os valores previstos 𝑦̂𝑃 seriam <td>$$𝑦̂𝑃=𝑋𝑃𝑤𝑃$$</td>.

3- O REQM (Raiz do Erro Quadrático Médio) é uma medida comum de qualidade para avaliar a eficácia de um modelo de regressão. Portanto, se o REQM for menor, significa que os valores previstos estão mais próximos dos valores reais, o que indica uma melhor qualidade da regressão linear.


**Prova analítica**

<td>
$$
w=(X^{T}X)^{-1}X^{T}y
$$

$$
w_P = [(XP)^{T}(XP)]^{-1}(XP)^{T}y
$$

$$
w_P = [(P^{T}X^{T})(XP)]^{-1}(XP)^{T}y,
$$

Usando a propriedade de reversividade da transposição de um produto de matrizes, assim conseguimos passar de $(XP)^{T}$ para $(P^{T}X^{T})$.

Após esse passo, podemos aplicar a inversa num produto matricial junto da propriedade associativa da multiplicação,

$$
w_P = [P^{T}(X^{T}X)P]^{-1}(XP)^{T}y = P^{-1}(X^{T}X)^{-1}(P^{T})^{-1}P^{T}(X^{T}y).
$$
    
Antes do termo $(X^{T}y)$ temos o termo $(P^{T})^{-1}P^{T}$, porém pela propriedade da identidade multiplicativa, temos que:
    
$$
(P^{T})^{-1}P^{T} = I,
$$

Continuando:
​
$$
w_P = P^{-1}(X^{T}X)^{-1}(P^{T})^{-1}P^{T}(X^{T}y) = P^{-1}(X^{T}X)^{-1}I(X^{T}y) = P^{-1}(X^{T}X)^{-1}(X^{T}y).
$$
    
Bem, se $w = (X^{T}X)^{-1}X^{T}y$, então:
    
$$
w_P = P^{-1}(X^{T}X)^{-1}(X^{T}y) = P^{-1}w,
$$
    
logo a matriz $A$ desconhecida é igual a $P^{-1}$ e:
    
$$
w_P = P^{-1}w.
$$
​
Para demonstrar de forma analítica que a ofuscação de dados não afeta a regressão linear, é suficiente substituir o valor de 𝑤𝑃
na equação a seguir pela relação previamente estabelecida:
    
$$
\hat{y_P} = (X_{val}P)w_P = X_{val}PP^{-1}w,
$$
    
pela propriedade da identidade multiplicativa, temos que:
​
$$
\hat{y_P} = X_{val}PP^{-1}w = X_{val}Iw = X_{val}w = \hat{y}.
$$
    
Assim, demonstramos que $\hat{y_P} = \hat{y}$, o que implica que a ofuscação de dados não tem efeito sobre a previsão realizada, embora possa modificar os coeficientes estimados. Como resultado, uma vez que a previsão permanece inalterada, nenhuma métrica de erro sofrerá modificação.
</td>

In [ ]:


def calculate_w(X, y):
    XTX_inv = np.linalg.inv(np.dot(X.T, X))
    w = np.dot(np.dot(XTX_inv, X.T), y)
    return w

def calculate_wP(XP, y):
    wP = np.dot(np.dot(np.linalg.inv(np.dot(XP.T, XP)), XP.T), y)
    return wP




XP = Xx + 1 #ex de dados ofuscados 

w = calculate_w(Xx, y)
wP = calculate_wP(XP, y)

print("w:", w)
print("wP:", wP)


In [ ]:
def predict_values(X, w):
    y_pred = np.dot(X, w)
    return y_pred

def predict_values_P(XP, wP):
    y_pred_P = np.dot(XP, wP)
    return y_pred_P

# Exemplo de uso
y_pred = predict_values(Xx, w)
y_pred_P = predict_values_P(XP, wP)

print("y_pred:", y_pred)
print("y_pred_P:", y_pred_P)


In [ ]:
def calculate_RMSE(y_true, y_pred):
    mse = np.mean((y_true - y_pred) ** 2)
    rmse = np.sqrt(mse)
    return rmse

# Exemplo de uso
RMSE_w = calculate_RMSE(y, y_pred)
RMSE_wP = calculate_RMSE(y, y_pred_P)

print("RMSE for w:", RMSE_w)
print("RMSE for wP:", RMSE_wP)


## Teste de regressão linear com ofuscação de dados

Agora, vamos provar que a Regressão Linear pode funcionar computacionalmente com a transformação de ofuscação escolhida.
Crie um procedimento ou uma classe que execute a Regressão Linear opcionalmente com a ofuscação. Você pode usar uma implementação pronta de Regressão Linear do scikit-learn ou sua própria.

Execute a Regressão Linear para os dados originais e os ofuscados, compare os valores previstos e os valores da métrica $R^2$ do REQM. Há alguma diferença?

**Procedimento**

- Crie uma matriz quadrada $P$ de números aleatórios.
- Verifique se é invertível. Caso contrário, repita o primeiro ponto até obtermos uma matriz invertível.
- <! seu comentário aqui!>
- Use $XP$ como a nova matriz de características

In [ ]:
def ev(y_true,y_pred):
    rmse = math.sqrt(sklearn.metrics.mean_squared_error(y_true, y_pred))
    print(f'REQM: {rmse:.2f}')

In [ ]:
G=df[['age', 'gender', 'income', 'family_members']].to_numpy()

In [ ]:
B=df_scaled[['age', 'gender', 'income', 'family_members']].to_numpy()

In [ ]:
rng = np.random.default_rng(seed=42)
Pw = rng.random(size=(G.shape[1], G.shape[1]))
Pw

In [ ]:
rng = np.random.default_rng(seed=42)
Pb = rng.random(size=(B.shape[1], B.shape[1]))
Pb

In [ ]:
oo = np.linalg.inv(Pw)

In [ ]:
bb=np.linalg.inv(Pb)

In [ ]:
mo=G@oo
mo

In [ ]:
mb=B@bb
mb

In [ ]:
x1 = mo
y1 = G


X_train1, X_test1, y_train1, y_test1 = train_test_split(x1, y1, test_size=0.3, random_state=12345)

lr1 = MyLinearRegression()

lr1.fit(X_train1, y_train1)
print(lr1.weights)

ev(y_train1, lr1.predict(X_train1))
ev(y_test1, lr1.predict(X_test1))


In [ ]:
lr1.fit(X_test1, y_test1)
ev(y_train1, lr1.predict(X_train1))
ev(y_test1, lr1.predict(X_test1))

In [ ]:
x2 = mb
y2 = B

X_train2, X_test2, y_train2, y_test2 = train_test_split(x2, y2, test_size=0.3, random_state=12345)

lr2 = MyLinearRegression()

lr2.fit(X_train2, y_train2)
print(lr2.weights)
ev(y_train2, lr2.predict(X_train2))
ev(y_test2, lr2.predict(X_test2))




In [ ]:
lr2.fit(X_test2, y_test2)
ev(y_train2, lr2.predict(X_train2))
ev(y_test2, lr2.predict(X_test2))

# Conclusões

* Não existe grupos muito claros, porém em um gráfico 3d da para ter uma noção melhor de como o dataframe se comporta

* Escalamento de dados melhora o desempenho do Knn, e normalmente de todos os testes. 

* A Prova que o REQM não muda com a ofuscação de dados foi feita.